# Dota 2 Data Collection

This is a collection of my Dota 2 stats using the Opendota API. <br>
Dota 2 is an online MOBA game that is very complex in nature with currently 123 heroes to select from. <br>
This project will be a summarization of different stats I collect about myself in the currently 1404 games I have played over the years. <br>

In [1]:
import config

import requests
import pandas as pd
import json

### Initialize our connection to the API

In [17]:
result = requests.get(f"https://api.opendota.com/api/matches/271145478?api_key={config.api_key}")
result.status_code

200

### Request our matches

In [27]:
matches = requests.get(f"https://api.opendota.com/api/players/{config.player_id}/matches")
matches_df = pd.read_json(matches.text)
matches_df

,match_id,player_slot,radiant_win,duration,game_mode,lobby_type,hero_id,start_time,version,kills,deaths,assists,skill,leaver_status,party_size
0,6439018464,4,True,2094,22,0,81,2022-02-19 08:45:42,21.0,4,4,10,NaN,0,2.0
1,6438932740,130,True,3398,22,0,99,2022-02-19 07:38:20,21.0,7,14,26,NaN,0,2.0
2,6435487456,2,False,3801,22,0,101,2022-02-17 04:19:15,NaN,18,13,8,NaN,0,NaN
3,6435452682,128,False,2241,22,0,54,2022-02-17 03:29:49,21.0,15,6,9,NaN,0,0.0
4,6433976001,2,False,2992,22,0,44,2022-02-16 05:03:37,21.0,17,6,8,NaN,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1399,1212863940,2,True,3333,1,0,35,2015-02-04 23:37:37,NaN,5,16,16,NaN,0,NaN
1400,1191146511,131,True,2452,1,0,35,2015-01-27 00:13:37,NaN,1,7,4,NaN,0,NaN
1401,866365677,129,True,1947,1,0,5,2014-08-31 03:09:05,NaN,0,5,0,NaN,0,NaN
1402,743658022,2,False,1575,1,0,67,2014-06-26 23:37:19,NaN,1,7,1,NaN,1,NaN


### Requesting the hero ID's so we can merge them with the matches dataframe.

In [38]:
heroes = requests.get("https://api.opendota.com/api/constants/heroes")
heroes_df = pd.read_json(heroes.text).T
heroes_df.rename(columns={'id':'hero_id', 'localized_name':'hero_name'}, inplace=True)
heroes_df[:5]

,hero_id,name,hero_name,primary_attr,attack_type,roles,img,icon,base_health,base_health_regen,...,str_gain,agi_gain,int_gain,attack_range,projectile_speed,attack_rate,move_speed,turn_rate,cm_enabled,legs
1,1,npc_dota_hero_antimage,Anti-Mage,agi,Melee,"[Carry, Escape, Nuker]",/apps/dota2/images/dota_react/heroes/antimage....,/apps/dota2/images/dota_react/heroes/icons/ant...,200,0.25,...,1.6,2.8,1.8,150,0,1.4,310,None,True,2
2,2,npc_dota_hero_axe,Axe,str,Melee,"[Initiator, Durable, Disabler, Jungler, Carry]",/apps/dota2/images/dota_react/heroes/axe.png?,/apps/dota2/images/dota_react/heroes/icons/axe...,200,2.75,...,3.4,2.2,1.6,150,900,1.7,310,None,True,2
3,3,npc_dota_hero_bane,Bane,int,Ranged,"[Support, Disabler, Nuker, Durable]",/apps/dota2/images/dota_react/heroes/bane.png?,/apps/dota2/images/dota_react/heroes/icons/ban...,200,0.25,...,2.5,2.5,2.5,400,900,1.7,305,None,True,4
4,4,npc_dota_hero_bloodseeker,Bloodseeker,agi,Melee,"[Carry, Disabler, Jungler, Nuker, Initiator]",/apps/dota2/images/dota_react/heroes/bloodseek...,/apps/dota2/images/dota_react/heroes/icons/blo...,200,0.25,...,2.7,3.1,2,150,900,1.7,300,None,True,2
5,5,npc_dota_hero_crystal_maiden,Crystal Maiden,int,Ranged,"[Support, Disabler, Nuker, Jungler]",/apps/dota2/images/dota_react/heroes/crystal_m...,/apps/dota2/images/dota_react/heroes/icons/cry...,200,0.25,...,2.2,1.6,3.3,600,900,1.7,280,None,True,2


In [39]:
matches_df = pd.merge(matches_df, heroes_df[['hero_id','hero_name']], how='left')

### Player wins arent explicitly given so we figure which side we're on and who won.

In [41]:
matches_df['player_side'] = "Radiant"
matches_df.loc[matches_df.player_slot >= 128, ['player_side']] = "Dire"

matches_df['player_win'] = True
matches_df.loc[(matches_df['player_side'] == "Dire") & (matches_df['radiant_win'] == True), ['player_win']] = False
matches_df.loc[(matches_df['player_side'] == "Radiant") & (matches_df['radiant_win'] == False), ['player_win']] = False
matches_df

,match_id,player_slot,radiant_win,duration,game_mode,lobby_type,hero_id,start_time,version,kills,deaths,assists,skill,leaver_status,party_size,hero_name,player_side,player_win
0,6439018464,4,True,2094,22,0,81,2022-02-19 08:45:42,21.0,4,4,10,NaN,0,2.0,Chaos Knight,Radiant,True
1,6438932740,130,True,3398,22,0,99,2022-02-19 07:38:20,21.0,7,14,26,NaN,0,2.0,Bristleback,Dire,False
2,6435487456,2,False,3801,22,0,101,2022-02-17 04:19:15,NaN,18,13,8,NaN,0,NaN,Skywrath Mage,Radiant,False
3,6435452682,128,False,2241,22,0,54,2022-02-17 03:29:49,21.0,15,6,9,NaN,0,0.0,Lifestealer,Dire,True
4,6433976001,2,False,2992,22,0,44,2022-02-16 05:03:37,21.0,17,6,8,NaN,0,0.0,Phantom Assassin,Radiant,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1399,1212863940,2,True,3333,1,0,35,2015-02-04 23:37:37,NaN,5,16,16,NaN,0,NaN,Sniper,Radiant,True
1400,1191146511,131,True,2452,1,0,35,2015-01-27 00:13:37,NaN,1,7,4,NaN,0,NaN,Sniper,Dire,False
1401,866365677,129,True,1947,1,0,5,2014-08-31 03:09:05,NaN,0,5,0,NaN,0,NaN,Crystal Maiden,Dire,False
1402,743658022,2,False,1575,1,0,67,2014-06-26 23:37:19,NaN,1,7,1,NaN,1,NaN,Spectre,Radiant,False


### Export

In [43]:
matches_df.to_csv("matches.csv")